# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

DATA_FOLDER = './Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [2]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

---


###### Reading of the metadata.xls file
The very first step is the reading of the metadata.xls file which indicates the column values of the expected final DataFrame. The pd.read_excel() function is used with the header keyword set to 0 in order to define the first row of the xls document as the column headers.

Finally the first rows of the metadata DataFrame are then printed

In [3]:
meta = pd.read_excel(DATA_FOLDER+'microbiome/metadata.xls', sheetname='Sheet1', header=0)
meta.head()

,BARCODE,GROUP,SAMPLE
0,MID1,EXTRACTION CONTROL,NaN
1,MID2,NEC 1,tissue
2,MID3,Control 1,tissue
3,MID4,NEC 2,tissue
4,MID5,Control 2,tissue


###### The read_microbiome function
The `read_microbiome(i, meta)` function is meant to read the microbiome excel file indexed with i. The meta DataFrame is passed as argument in order to create the columns of the ith microbiome DataFrame. The function returns 2 different versions: one with MultiIndexing and another one that keeps the columns defined in the metadata.xls file but provide a unique index for the rows.

1. The MultiIndexed DataFrame for the ith microbiome is output as midi_multi_index. In midi_multi_index, 'NAME', 'BARCODE', 'GROUP' and 'SAMPLE' are defined as the MultiIndices.

2. The midi_unique DataFrame is the DataFrame with unique index that keeps the columns defined in metadata.xls as proper DataFrame columns. In midi_unique the index_col is renamed as 'id'. For the index to be unique it is built from the concatenation of the following columns: 'NAME', 'BARCODE', 'GROUP', 'SAMPLE'.

It's up to the user to choose which format he/she prefers. But it is strongly advised to use the MultiIndexed DataFrame output as midi_multi_index because it is much more elegant.

Finally note that by default, every NaN value is replaced with the string 'unknown' using the `fillna()` DataFrame method.





In [4]:
def read_microbiome(i, meta):
    midi = pd.read_excel(DATA_FOLDER+'microbiome/MID' + str(i) + '.xls', sheetname='Sheet 1', header=None)
    midi.columns=['NAME', 'VALUE']
    for j in range(3):
        midi[meta.columns[j]] = meta[meta.columns[j]][i-1]
    midi = midi.fillna('unknown')
    
    # midi_unique construction
    midi_unique = midi.copy()
    midi_unique.index = midi.NAME + midi.BARCODE + midi.GROUP + midi.SAMPLE
    midi_unique.index.name = 'ID'
    # Reordering of the columns
    midi_unique = midi_unique[['NAME', 'BARCODE', 'GROUP', 'SAMPLE', 'VALUE']]
    
    # midi_multi_index construction
    midi_multi_index = midi.set_index(['NAME'] + [x for x in meta.columns])

    return (midi_multi_index, midi_unique)

Then the `read_microbiome()` function is used to read all the microbiome files through a for loop. 9 DataFrames are processed and each of them is converted in a MultiIndexed DataFrame and in more classical DataFrame keeping its columns. The MultiIndexed DataFrames are stored in the mid_multi_index list and the other one in the mid_unique list.

In [63]:
mid_multi_index = []
mid_unique = []
for i in range(1,10):
    result = read_microbiome(i, meta)
    mid_multi_index += [result[0]]
    mid_unique += [result[1]]

###### DataFrame concatenation
Here the 9 original DataFrames are concatenated along the rows using an outer join.
It is carried out for both types of DataFrame that we are producing here, that are: mid_multi_index and mid_unique

In [64]:
concat_mid_multi_index = pd.concat(mid_multi_index, axis = 0) # Preferred version
concat_mid_unique = pd.concat(mid_unique, axis = 0) # Version with columns kept as proper columns

In [65]:
concat_mid_multi_index.head()

,,,,VALUE
NAME,BARCODE,GROUP,SAMPLE,
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera",MID1,EXTRACTION CONTROL,unknown,7
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrolobus",MID1,EXTRACTION CONTROL,unknown,2
"Archaea ""Crenarchaeota"" Thermoprotei Sulfolobales Sulfolobaceae Stygiolobus",MID1,EXTRACTION CONTROL,unknown,3
"Archaea ""Crenarchaeota"" Thermoprotei Thermoproteales Thermofilaceae Thermofilum",MID1,EXTRACTION CONTROL,unknown,3
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanocellales Methanocellaceae Methanocella",MID1,EXTRACTION CONTROL,unknown,7


In [66]:
concat_mid_unique.head()

,NAME,BARCODE,GROUP,SAMPLE,VALUE
ID,,,,,
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae IgnisphaeraMID1EXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",MID1,EXTRACTION CONTROL,unknown,7
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae PyrolobusMID1EXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",MID1,EXTRACTION CONTROL,unknown,2
"Archaea ""Crenarchaeota"" Thermoprotei Sulfolobales Sulfolobaceae StygiolobusMID1EXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",MID1,EXTRACTION CONTROL,unknown,3
"Archaea ""Crenarchaeota"" Thermoprotei Thermoproteales Thermofilaceae ThermofilumMID1EXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",MID1,EXTRACTION CONTROL,unknown,3
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanocellales Methanocellaceae MethanocellaMID1EXTRACTION CONTROLunknown","Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",MID1,EXTRACTION CONTROL,unknown,7


###### Proof of concept
Here we show that the indexing for both DataFrames is unique.

In [67]:
concat_mid_multi_index.index.is_unique

True

In [68]:
concat_mid_unique.index.is_unique

True

Then we show that no NaN values are indeed present in neither of the DataFrames since they all have been replaced by 'unknown'.

In [69]:
True in concat_mid_multi_index.isnull()

False

In [70]:
True in concat_mid_unique.isnull()

False

---

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [5]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [6]:
# Write your answer here